 TODOs (intentionally not implemented):
- `build_training_data`
- `class PINN`
- `model_loss`
- `train`


In [22]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import math
from typing import Tuple

import numpy as np
import torch
import torch.nn as nn


def build_training_data():
    """
    TODO : Implement the training-data construction.
    """

    raise NotImplementedError("TODO: implement build_training_data")


class PINN(nn.Module):
    def __init__(self, hidden_size = 64, n_layers=4):
        super().__init__()
        layers: list[nn.Module] = [nn.Linear(1, hidden_size), nn.Tanh()]
        for _ in range(n_layers - 1):
            layers += [nn.Linear(hidden_size, hidden_size), nn.Tanh()]
        layers.append(nn.Linear(hidden_size, 4))
        self.net = nn.Sequential(*layers)

        self.raw_xf = nn.Parameter(torch.tensor(0.0)) # sigmoid  
        self.raw_Zf_r = nn.Parameter(torch.tensor(0.0)) # softplus 
        self.raw_Zf_i = nn.Parameter(torch.tensor(0.0)) # reactance

        raw_R = nn.Parameter(torch.tensor(0.0))
        raw_X =nn.Parameter(torch.tensor(0.0))
        raw_G = nn.Parameter(torch.tensor(0.0))
        raw_B = nn.Parameter(torch.tensor(0.0)) 
        
        
    def forward(self, x):
        out = self.net(x)
        return out[:, 0:1], out[:, 1:2],out[:, 2:3], out[:, 3:4]
    
    def fault_position(self):
        return torch.sigmoid(self.raw_xf)
    
    def fault_impedance(self):
        Zf_r = nn.functional.softplus(self.raw_Zf_r)
        Zf_i = self.raw_Zf_i
        return Zf_r, Zf_i 
    
    def get_line_parameters(self):
        softpl = nn.functional.softplus
        return softpl(self.raw_R), softpl(self.raw_X), softpl(self.raw_G),softpl(self.raw_B)
    
    def gaussian(self,x):
        xf  = self.fault_position()
        sig = 0.04 
        return torch.exp(-0.5 * ((x - xf) / sig) ** 2) / (sig * (2 * 3.14159265) ** 0.5)
        



def grad(outputs, inputs):  # compute derivative of outputs w.r.t. inputs 
    return torch.autograd.grad( 
        outputs,
        inputs,
        grad_outputs=torch.ones_like(outputs), 
        create_graph= True,  
        retain_graph=True,
    )[0]

def pde(x, model): 
    Rv, Iv, Ri, Ii = model(x) 
    dRv = grad(Rv, x)
    dIv = grad(Iv, x)
    dRi = grad(Ri, x)
    dIi = grad(Ii, x)

    R_n, X_n, G_n, B_n = model.get_line_parameters()

    # Voltage and current phasor PDE residuals
    r1 = dRv + R_n*Ri - X_n*Ii
    r2 = dIv + R_n*Ii + X_n*Ri

    xf = model.fault_position()  
    # ? why not Ri_f, Ii_f 
    Rv_f, Iv_f, _ , _  = model(xf)  # 
    Zf_r, Zf_i = model.fault_impedance()  
    Zf2 = Zf_r**2 + Zf_i**2 + 1e-12

    Re_If = Rv_f*Zf_r/Zf2 + Iv_f*Zf_i/Zf2
    Im_If = Iv_f*Zf_r/Zf2 - Rv_f*Zf_i/Zf2

    gauss = model.gaussian(x)  #

    r3 = dRi + G_n*Rv - B_n*Iv + Re_If*gauss
    r4 = dIi + G_n*Iv + B_n*Rv + Im_If*gauss

    return r1,r2,r3,r4 



    
    
    
    
    raise NotImplementedError("TODO: implement PDE residual computation")






def model_loss(
       ):
    """
    TODO : Implement PINN loss.

    """
    raise NotImplementedError("TODO: implement model_loss")






In [ ]:
# Run after implementing all TODOs:
#model = train()
#evaluate(model)
